In [25]:
import tequila as tq
import warnings
import numpy as np
warnings.filterwarnings("ignore")

In [27]:
# Lets look at the Tapered Binary transformation
print("\n -----------TAPERING----------- \n ")

g = "h 0.0 0.0 0.0\nh 0.0 0.0 1.5\nh 0.0 0.0 3.0\nh 0.0 0.0 4.5"

mol = tq.Molecule(backend="pyscf", geometry=g, basis_set="sto-3g", transformation="TaperedBinary").use_native_orbitals()

# You can check the Hartree Fock Energy and FCI energy like this:
print('HF ', mol.compute_energy('HF'))
fci = mol.compute_energy('FCI')
print('FCI ', fci)


SPA = mol.make_ansatz("SPA", edges=[(0,1),(2,3)])
H = mol.make_hamiltonian()

# IS THIS G1?
# This is how to properly calculate the SPA energy, without optimizing the orbitals
# The UR0 basically optimizes the orbitals
U0 = mol.UR(0,1,'a') + mol.UR(2,3,'b')
U = U0 + SPA + U0.dagger()
U_cirq = tq.compile(U, backend="cirq")
res = tq.minimize(tq.ExpectationValue(H=H,U=U),silent=True)
print(" ----------------------- SPA (G1) ----------------------- ")
print(f"Energy: {res.energy}")
print(f"difference from fci: {abs(res.energy-fci)*1000} meh?")
print(U)
print(U_cirq.circuit)
print(f"Measure Hamiltonian H = {H}\n")
try:
    U.export_to(filename="h4_ground_state_tapered_G1.pdf")
except Exception as E:
    print("no qpic and/or latex installed ... sorry")

# THIS JUST CLASSICALLY OPTIMIZES THE ORBITALS?
# If u want regular obrital optimization u can use:
guess = np.eye(4)
opt = tq.quantumchemistry.optimize_orbitals(molecule=mol,circuit=SPA, initial_guess=guess, silent=True).molecule
H_opt = opt.make_hamiltonian()
res = tq.minimize(tq.ExpectationValue(H=H_opt,U=SPA), silent=True)
SPA_cirq = tq.compile(SPA, backend="cirq")
print("classically optimized orbitals")
print(f"Energy: {res.energy}")
print(f"difference from fci: {abs(res.energy-fci)*1000} meh?\n")
print(SPA_cirq.circuit)
print(f"Measure Hamiltonian H = {H_opt}\n")
try:
    SPA.export_to(filename="h4_ground_state_tapered_G1_classically_optimized_orbitals.pdf")
except Exception as E:
    print("no qpic and/or latex installed ... sorry")

# If u want to use Orbital Correlator for more graphs eg. (1,2) u can use:
U1 = mol.UR(1,2,'c')
UC = mol.UC(1,2,'d')
U_p = SPA + U0 + U1 + UC + U0.dagger() + U1.dagger()
res = tq.minimize(tq.ExpectationValue(H=H,U=U_p), silent=True)
U_p_cirq = tq.compile(U_p, backend="cirq")
print("SPA + Correlator (G2)")
print(f"Energy: {res.energy}")
print(f"difference from fci: {abs(res.energy-fci)*1000} meh?\n")
print(U_p_cirq.circuit)
print(f"Measure Hamiltonian H = {H}\n")
try:
    U_p.export_to(filename="h4_ground_state_tapered_G2.pdf")
except Exception as E:
    print("no qpic and/or latex installed ... sorry")

# U can also add Orbital optimization to the Rotator - Corellator / use our optimized Hamiltonian
U_plus = SPA
U_plus += mol.UR(0,2, angle=(tq.Variable("a_1") + 0.5) * np.pi) + mol.UR(1,3, angle=(tq.Variable("a_2") + 0.5) * np.pi)
U_plus += mol.UC(0,2, angle=tq.Variable("b_1") * np.pi) + mol.UC(1,3, angle=tq.Variable("b_2") * np.pi)
U_plus += mol.UR(0,2, angle=(tq.Variable("c_1") + 0.5) * np.pi) + mol.UR(1,3, angle=(tq.Variable("c_2") + 0.5) * np.pi)
res = tq.minimize(tq.ExpectationValue(H=H_opt,U=U_plus),silent=True)
U_plus_cirq = tq.compile(U_plus, backend="cirq")
print("SPA + Correlator (G2) + classically optimized orbitals")
print(f"Energy: {res.energy}")
print(f"difference from fci: {abs(res.energy-fci)*1000} meh?")
print(U_plus_cirq.circuit)
print(f"Measure Hamiltonian H = {H_opt}\n")
# try:
#     U.export_to(filename="overlap_circuit.pdf")
# except Exception as E:
#     print("no qpic and/or latex installed ... sorry")


 -----------TAPERING----------- 
 
HF  -1.8291374124430182
FCI  -1.9961503255188089
 ----------------------- SPA (G1) ----------------------- 
Energy: -1.9562546212337832
difference from fci: 39.895704285025644 meh?
circuit: 
FermionicExcitation(target=(0, 1, 2), control=(), parameter=a)
FermionicExcitation(target=(1, 2, 3), control=(), parameter=a)
FermionicExcitation(target=(0, 2, 4, 5), control=(), parameter=b)
FermionicExcitation(target=(0, 1, 2, 3, 4, 5), control=(), parameter=b)
X(target=(0,))
Ry(target=(2,), parameter=((0, 1), 'D', None))
X(target=(0,), control=(2,))
X(target=(4,))
Ry(target=(4,), parameter=((2, 3), 'D', None))
X(target=(1,), control=(0,))
X(target=(3,), control=(2,))
X(target=(5,), control=(4,))
FermionicExcitation(target=(0, 1, 2, 3, 4, 5), control=(), parameter=f([b]))
FermionicExcitation(target=(0, 2, 4, 5), control=(), parameter=f([b]))
FermionicExcitation(target=(1, 2, 3), control=(), parameter=f([a]))
FermionicExcitation(target=(0, 1, 2), control=(), par

In [34]:
# Now everything without tapering to compare:
print("\n -----------NO TAPERING----------- \n ")

mol = tq.Molecule(backend="pyscf", geometry=g, basis_set="sto-3g", transformation="JordanWigner").use_native_orbitals()

# You can check the Hartree Fock Energy and FCI energy like this:
print('HF ', mol.compute_energy('HF'))
fci = mol.compute_energy('FCI')
print('FCI ', fci)

SPA = mol.make_ansatz("SPA", edges=[(0,1),(2,3)])
H = mol.make_hamiltonian()

# IS THIS G1?
# This is how to properly calculate the SPA energy, without optimizing the orbitals
# The UR0 basically optimizes the orbitals
U0 = mol.UR(0,1,'a') + mol.UR(2,3,'b')
U = U0 + SPA + U0.dagger()
U_cirq = tq.compile(U, backend="cirq")
res = tq.minimize(tq.ExpectationValue(H=H,U=U),silent=True)
print("SPA (G1)")
print(f"Energy: {res.energy}")
print(U)
print(U_cirq.circuit)
print(f"difference from fci: {abs(res.energy-fci)*1000} meh?\n")
try:
    U.export_to(filename="h4_ground_state_JW_G1.pdf")
except Exception as E:
    print("no qpic and/or latex installed ... sorry")

# THIS JUST CLASSICALLY OPTIMIZES THE ORBITALS?
# If u want regular obrital optimization u can use:
guess = np.eye(4)
opt = tq.quantumchemistry.optimize_orbitals(molecule=mol,circuit=SPA, initial_guess=guess, silent=True).molecule
H_opt = opt.make_hamiltonian()
res = tq.minimize(tq.ExpectationValue(H=H_opt,U=SPA), silent=True)
print("classically optimized orbitals")
print(f"Energy: {res.energy}")
print(f"difference from fci: {abs(res.energy-fci)*1000} meh?\n")



# If u want to use Orbital Correlator for more graphs eg. (1,2) u can use:
U1 = mol.UR(1,2,'c')
UC = mol.UC(1,2,'d')
U_p = SPA + U0 + U1 + UC + U0.dagger() + U1.dagger()
res = tq.minimize(tq.ExpectationValue(H=H,U=U_p), silent=True)
print("SPA + Correlator (G2)")
print(f"Energy: {res.energy}")
print(f"difference from fci: {abs(res.energy-fci)*1000} meh?\n")


# U can also add Orbital optimization to the Rotator - Corellator / use our optimized Hamiltonian
U_plus = SPA
U_plus += mol.UR(0,2, angle=(tq.Variable("a_1") + 0.5) * np.pi) + mol.UR(1,3, angle=(tq.Variable("a_2") + 0.5) * np.pi)
U_plus += mol.UC(0,2, angle=tq.Variable("b_1") * np.pi) + mol.UC(1,3, angle=tq.Variable("b_2") * np.pi)
U_plus += mol.UR(0,2, angle=(tq.Variable("c_1") + 0.5) * np.pi) + mol.UR(1,3, angle=(tq.Variable("c_2") + 0.5) * np.pi)
res = tq.minimize(tq.ExpectationValue(H=H_opt,U=U_plus),silent=True)
print("SPA + Correlator (G2) + classically optimized orbitals")
print(f"Energy: {res.energy}")
print(f"difference from fci: {abs(res.energy-fci)*1000} meh?")


 -----------NO TAPERING----------- 
 
HF  -1.8291374124430186
FCI  -1.9961503255188089
SPA (G1)
Energy: -1.9562546212337784
circuit: 
FermionicExcitation(target=(0, 1, 2), control=(), parameter=a)
FermionicExcitation(target=(1, 2, 3), control=(), parameter=a)
FermionicExcitation(target=(4, 5, 6), control=(), parameter=b)
FermionicExcitation(target=(5, 6, 7), control=(), parameter=b)
X(target=(0,))
Ry(target=(2,), parameter=((0, 1), 'D', None))
X(target=(0,), control=(2,))
X(target=(4,))
Ry(target=(6,), parameter=((2, 3), 'D', None))
X(target=(4,), control=(6,))
X(target=(1,), control=(0,))
X(target=(3,), control=(2,))
X(target=(5,), control=(4,))
X(target=(7,), control=(6,))
FermionicExcitation(target=(5, 6, 7), control=(), parameter=f([b]))
FermionicExcitation(target=(4, 5, 6), control=(), parameter=f([b]))
FermionicExcitation(target=(1, 2, 3), control=(), parameter=f([a]))
FermionicExcitation(target=(0, 1, 2), control=(), parameter=f([a]))

                                          

In [ ]:
print("------------------------------")

# If u want to access the values from the minimization u can do:

print("variables of minimization:\n", res.variables)   

# If u want to simulate with ur variables do:

print("energy of simulation with variables: ", tq.simulate(tq.ExpectationValue(H=H_opt, U=U_plus), res.variables))


------------------------------
variables of minimization:
 ((0, 1), 'D', None) : -0.6917573992021984
((2, 3), 'D', None) : -0.6917444855513156
a_1 : 0.00015366980479949104
a_2 : 4.237018712519192e-05
b_1 : 0.09636982107133968
b_2 : -0.14644217744270446
c_1 : 0.00015287119828524275
c_2 : 4.003177451213307e-05

energy of simulation with variables:  -1.9864531968718857
